In [ ]:
class CustomDataset(Dataset):
    def __init__(self, path, subset='train', transform=None):
        
        self.path = path
        self.folder = folder
        self.transform = transform
        
        self.image_paths = []
        self.labels = []

        folder_path = os.path.join(path, subset)
        
        if os.path.isdir(folder_path):
            for class_name in os.listdir(folder_path):
                dir_path = os.path.join(folder_path, class_name)
                
                if os.path.isdir(dir_path):
                    label = class_name  # 폴더명을 레이블로 사용
                    for filename in os.listdir(dir_path):
                        if filename.endswith('.png'):
                            self.image_paths.append(os.path.join(dir_path, filename))
                            self.labels.append(label)
        else:
            raise ValueError(f'Invalid subset: {subset}')

            
    def __len__(self):
        return len(self.image_paths)

    
    def __getitem__(self, idx):
        image_path = self.image_paths[idx]
        label = self.labels[idx]
        
        image = Image.open(image_path)

        if self.transform:
            image = self.transform(image)

        return image, label

In [ ]:
class Dataloader():
    def __init__(self, path):
        normalize = transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
        
        transform_train = transforms.Compose([
                        transforms.Resize((512, 512)),
                        transforms.ToTensor(),
                        normalize
                        ])
        transform_test = transforms.Compose([
                        transforms.Resize((512, 512)),
                        transforms.ToTensor(),
                        normalize
                        ])
        
        trainset = CustomDataset(path, subset='train', transform=transform_train)
        valset = CustomDataset(path, subset='val', transform=transform_train)
        testset = CustomDataset(path, subset='test', transform=transform_test)
        
        self.train_loader = DataLoader(trainset, batch_size=64, shuffle=True, num_workers=4)
        self.val_loader = DataLoader(valset, batch_size=64, shuffle=True, num_workers=4)
        self.test_loader = DataLoader(testset, batch_size=64, shuffle=True, num_workers=4)
        
    def getloader(self):
        return self.train_loader, self.val_loader, self.test_loader


In [ ]:
# 모델 훈련
num_epochs = 10
for epoch in range(num_epochs):
    running_loss = 0.0
    correct_predictions = 0
    total_predictions = 0
    
    for images, labels in train_dataloader:
        images = images.to(device)
        labels = labels.to(device)
        
        # 순전파
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # 역전파 및 최적화
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        # 정확도 계산
        _, predicted = torch.max(outputs, 1)
        correct_predictions += (predicted == labels).sum().item()
        total_predictions += labels.size(0)
        
        # 통계 출력
        running_loss += loss.item() * images.size(0)
    
    epoch_loss = running_loss / len(train_dataset)
    epoch_accuracy = correct_predictions / total_predictions * 100
    
    print(f'Epoch {epoch+1}/{num_epochs} - Loss: {epoch_loss:.4f} - Accuracy: {epoch_accuracy:.2f}%')